In [3]:
{-# LANGUAGE OverloadedStrings #-}
import System.IO
import Control.Monad
import Data.List.Split
import Data.Bool
import Data.Set (Set)
import qualified Data.Text as T
import qualified Data.Text.IO as TIO
import qualified Data.Set as Set
import Text.Regex.Posix
import qualified Data.Map as Map
import Debug.Trace
import qualified Data.Attoparsec.Text as A
import Control.Applicative
import Data.Either

Line 1: Unused LANGUAGE pragma
Found:
{-# LANGUAGE OverloadedStrings #-}
Why not:

In [73]:
parseData f = do 
 d <- readFile "input"
 f d

In [96]:
data Op = Jmp Int | Nop Int | Acc Int

In [97]:
runOp (Jmp n) (ip, acc) = (ip + n, acc)
runOp (Nop n) (ip, acc) = (ip + 1, acc)
runOp (Acc n) (ip, acc) = (ip + 1, acc + n)

In [98]:
runProgram program ips (ip, acc) =
  let
    instruction = if ip < length program then Just (program !! ip) else Nothing
    ip' = case Map.lookup ip ips of
        Just _ -> Map.adjust (+1) ip ips
        Nothing -> Map.insert ip 1 ips
  in
    case instruction of
      Just i -> case Map.lookup ip ips of
        Just _ -> ("inf", acc)
        Nothing -> runProgram program ip' (runOp i (ip, acc))
      Nothing -> ("term", acc)

In [113]:
parseNOP = do
    A.string "nop"
    A.space
    n <- A.signed A.decimal
    return (Nop n)
    
parseJMP = do
    A.string "jmp"
    A.space
    n <- A.signed A.decimal
    return (Jmp n)

parseACC = do
    A.string "acc"
    A.space
    n <- A.signed A.decimal
    return (Acc n)

In [114]:
parseInstruction = do
  instruction <- parseNOP <|> parseJMP <|> parseACC
  A.char '\n'
  return instruction

In [115]:
parseProgram = A.many1 parseInstruction

In [116]:
changeOp (Jmp n) = Nop n
changeOp (Nop n) = Jmp n
changeOp (Acc n) = Acc n

In [117]:
replace :: [a] -> Int -> (a -> a) -> [a]
replace xs i op = case splitAt i xs of
   (before, old:after) -> before ++ op old: after
   _ -> xs

In [118]:
changeInstruction program i = replace program i changeOp

In [119]:
f d = 
  let
    program = fromRight [] $ A.parseOnly parseProgram (T.pack d)
    changePos = [0..length program - 1]
    programs = map (changeInstruction program) changePos
    results = map (\program -> runProgram program Map.empty (0,0)) programs
    isTerm ("term", v) = True
    isTerm ("inf", v) = False
  in
    do
        print $ runProgram program Map.empty (0, 0)
        print $ filter isTerm results
    

parseData f

("inf",1087)
[("term",780)]